In [1]:
from helper_functions import *
import plotly.graph_objects as go
import pandas as pd
import wandb



In [2]:

from datetime import datetime
now = datetime.now()
datetime_str = now.strftime("%m/%d_%H:%M:%S")
datetime_str

'01/22_20:33:36'

In [3]:
kris_df = preprocess_data(pd.read_pickle("Kris_merged_yolov8.pkl"))
varb_df = preprocess_data(pd.read_pickle("Varb_merged_yolov8.pkl"))
slakt_df = preprocess_data(pd.read_pickle("Slaug_merged_yolov8.pkl"))

datasets = {'kris':kris_df,
            'varb':varb_df,
            'slakt': slakt_df}

In [4]:
# import plotly.graph_objects as go

# # List of datasets
# datasets = [kris_df, varb_df, slakt_df]
# dataset_names = ['kris_df', 'varb_df', 'slakt_df']  # Names for labeling

# # Iterate over datasets
# for i, data_df in enumerate(datasets):
#     # Plot 1: Distribution of car_counts
#     fig = go.Figure()
#     fig.add_trace(go.Histogram(x=data_df["car_count"], nbinsx=40, name="car_counts"))
#     fig.update_layout(title_text=f"Distribution of car_counts in {dataset_names[i]}", xaxis_title="cars_detected", yaxis_title="frequency")
#     fig.show()

#     # Plot 2: car_counts by Day of Week
#     fig = go.Figure()
#     fig.add_trace(go.Box(x=data_df["day_of_week_name"], y=data_df["car_count"], name="car_counts by Day of Week"))
#     fig.update_layout(title_text=f"car_counts by Day of Week in {dataset_names[i]}", xaxis_title="Day of Week", yaxis_title="car_count")
#     fig.show()

#     # Plot 3: car_counts by Hour of Day
#     fig = go.Figure()
#     fig.add_trace(go.Box(x=data_df["hour_of_day"], y=data_df["car_count"], name="car_counts by Hour of Day"))
#     fig.update_layout(title_text=f"car_counts by Hour of Day in {dataset_names[i]}", xaxis_title="Hour of Day", yaxis_title="car_count")
#     fig.show()

#     # Plot 4: car_counts by Day of Month
#     fig = go.Figure()
#     fig.add_trace(go.Scatter(x=data_df["day"], y=data_df["car_count"], mode="markers", name="car_counts by Day of Month"))
#     fig.update_layout(title_text=f"car_counts by Day of Month in {dataset_names[i]}", xaxis_title="Day of Month", yaxis_title="car_count")
#     fig.show()

#     # Plot 5: Trend over Timestamp for car_count
#     trace = go.Scatter(x=data_df.index, y=data_df["car_count"], mode="lines", name="car_count")
#     layout = go.Layout(title=f"Trend over time for car_count in {dataset_names[i]}", xaxis=dict(title="Timestamp"), yaxis=dict(title="car_count"))
#     fig = go.Figure(data=[trace], layout=layout)
#     fig.show()


In [5]:
from sklearn.gaussian_process.kernels import RBF, WhiteKernel, ExpSineSquared, ConstantKernel, Exponentiation as C
from sklearn.gaussian_process.kernels import RBF, WhiteKernel, ExpSineSquared, ConstantKernel as C
from sklearn.gaussian_process import GaussianProcessRegressor
# RBF kernel with a length scale of 1.0
k1 = 1 * RBF(length_scale=0.5, length_scale_bounds=(1e-3, 1e3))

# White kernel to accar_count for noise
k2 = WhiteKernel(noise_level=1, noise_level_bounds=(1e-10, 1e+1))

# Periodic kernel
k3 = ExpSineSquared(length_scale=1.0,
                    length_scale_bounds=(1e-1, 50) ,
                    periodicity=1,
                    periodicity_bounds= (1e-1, 50))
# locally periodic
k4 = k1 * k3

In [6]:
# rbf with noise and periodicity C0
rbf_noise_per= k1 + k2 + k3 # this is not really needed

# locally periodic with WN C1
loc_per = k4 + k2

#noisy locally periodic C2
noise_loc_per = k4 *k2

#locally periodic, global periodic with WN C3
loc_glo_per_wn = k4 + k3 + k2

#Locally noisy periodic kernel with added global periodicity C4
noisy_lp_gp = (k4+ k2)+k3
# kernels = [rbf_noise_per, loc_per, noise_loc_per, loc_glo_per_wn]

# Creating the dictionary
# kernels = {
#     "rbf_noise_per": rbf_noise_per,
#     "loc_per": loc_per,
#     "noise_loc_per": noise_loc_per,
#     "loc_glo_per_wn": loc_glo_per_wn
# }

kernels = {"noisy_lp_gp": noisy_lp_gp}


In [7]:
features = {'hour':'hour_of_day', 
            'week':'day_of_week',
            'month':'day'}

In [8]:
# define splits



In [9]:

# Iterate through each dataset
for dataset_name, data in datasets.items():
    # Iterate through each kernel
    train, valid, test = split_data(data, train_ratio= 0.6, valid_ratio= 0.2)
    
    for kernel_name, kernel in kernels.items():
        # Initialize the Gaussian Process Regressor with the current kernel
        gpr = GaussianProcessRegressor(kernel=kernel)

        # Iterate through each feature
        for feature_name, feature_data in features.items():
            
            if wandb.run is not None:
                wandb.run.finish()
                
            # Initialize a wandb run for tracking
            run = wandb.init(project="Merged_GPR_Experiments", 
                             name=f"Single_win_{dataset_name}_{kernel_name}:{feature_name}")
            try:
                
                # Set experiment configuration
                config = wandb.config
                config.dataset = dataset_name
                config.kernel = kernel_name
                config.feature = feature_name

                # Call your experiment function
                y_test_pred, y_test_std, gpr = single_window_gpr_process(
                    run,
                    train,
                    valid,
                    test,
                    gpr,
                    features=feature_data,
                    target='centered_car_count',  # Adjust as needed
                    agg_func='mean'
                )
            except Exception as e:
                print(f"An error occurred: {e}")
            finally:
                run.finish()
            

      

print('All experiments completed.')


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: afn-sy. Use `wandb login --relogin` to force relogin


/home/sf_afn/usr_sa/anaconda3/envs/prkng_v2_env/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:429: ConvergenceWarning:

The optimal value found for dimension 0 of parameter k1__k1__k1__k2__length_scale is close to the specified upper bound 1000.0. Increasing the bound and calling fit again may find a better value.



wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


LML_test,▁
LML_train,▁▁
LML_validate,▁
MAE_test,▁
MAE_train,▁▁
MAE_validate,▁
MAPE_test,▁
MAPE_train,▁▁
MAPE_validate,▁
MSE_test,▁
MSE_train,▁▁


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


...

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


LML_test,▁
LML_train,▁▁
LML_validate,▁
MAE_test,▁
MAE_train,▁▁
MAE_validate,▁
MAPE_test,▁
MAPE_train,▁▁
MAPE_validate,▁
MSE_test,▁
MSE_train,▁▁


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


LML_test,▁
LML_train,▁▁
LML_validate,▁
MAE_test,▁
MAE_train,▁▁
MAE_validate,▁
MAPE_test,▁
MAPE_train,▁▁
MAPE_validate,▁
MSE_test,▁
MSE_train,▁▁


All experiments completed.


In [10]:

# Iterate through each dataset
for dataset_name, data in datasets.items():
    # Iterate through each kernel
    train,valid ,test = split_data(data, train_ratio= 0.8,valid_ratio=0)
    
    for kernel_name, kernel in kernels.items():
        # Initialize the Gaussian Process Regressor with the current kernel
        gpr = GaussianProcessRegressor(kernel=kernel)

        # Iterate through each feature
        for feature_name, feature_data in features.items():
            if wandb.run is not None:
                wandb.run.finish()
                
            run = wandb.init(project="Merged_GPR_Experiments",
                             name=f"Sliding_{dataset_name}_{kernel_name}:{feature_name}") 

            try:
                # Initialize a wandb run for tracking
                config = wandb.config
                # config.ini_train_win_len = 24
                config.ini_train_win_len = 24 * 7
                config.valid_win_len = 24
                # config.valid_win_len = 24 *7
                config.step_size = 1 # 24 hours or 1 day

                # Call the expanding window function
                y_pred, y_std, gpr = sliding_window_gpr_process(run,
                                                                train,
                                                                gpr,
                                                                test, 
                                                                train_win_len=config.ini_train_win_len, 
                                                                valid_win_len=config.valid_win_len,
                                                                features=feature_data,
                                                                target='centered_car_count',
                                                                agg_func='mean')
            except Exception as e:
                print(f"An error occurred: {e}")
            finally:
                run.finish()


print('All experiments completed.')


Processing sliding window:   0%|                                            | 0/108 [00:00<?, ?it/s]/home/sf_afn/usr_sa/anaconda3/envs/prkng_v2_env/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:429: ConvergenceWarning:

The optimal value found for dimension 0 of parameter k1__k1__k1__k2__length_scale is close to the specified upper bound 1000.0. Increasing the bound and calling fit again may find a better value.

Processing sliding window:   2%|▋                                   | 2/108 [00:00<00:24,  4.37it/s]

Step: 1


Processing sliding window:   4%|█▎                                  | 4/108 [00:01<00:26,  3.90it/s]/home/sf_afn/usr_sa/anaconda3/envs/prkng_v2_env/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:429: ConvergenceWarning:

The optimal value found for dimension 0 of parameter k1__k1__k1__k2__length_scale is close to the specified upper bound 1000.0. Increasing the bound and calling fit again may find a better value.

/home/sf_afn/usr_sa/anaconda3/envs/prkng_v2_env/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:429: ConvergenceWarning:

The optimal value found for dimension 0 of parameter k1__k2__noise_level is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.



Step: 2


Processing sliding window:   6%|██                                  | 6/108 [00:01<00:27,  3.72it/s]/home/sf_afn/usr_sa/anaconda3/envs/prkng_v2_env/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:429: ConvergenceWarning:

The optimal value found for dimension 0 of parameter k1__k1__k1__k2__length_scale is close to the specified upper bound 1000.0. Increasing the bound and calling fit again may find a better value.



...

wandb: Network error (ConnectionError), entering retry loop.
Processing sliding window: 100%|██████████████████████████████████| 108/108 [02:00<00:00,  1.11s/it]


Step: 54
Testing final GPR on test set


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


LML_test,▁
LML_train,▁▁▁▃▄▄▅▆▄▃▅▇███▆▆▆▅▅▇▆▆▄▅▅▇▅▃▃▄▃█▆▄▅▄▃▂▃
LML_validate,▁▁▁▃▄▄▅▅▅▃▅▆▇██▆▆▆▅▅▇▆▆▄▆▅▅▇▃▃▃▂▃██▄▅▄▃▃
MAE_test,▁
MAE_train,▆▁█▁▄▃▁▅▆▇▆▅▄▃▄▄▅▅▆▁▁▃▄▅▃▁▄▆▇█▅▄▁▄▁▂▅▂▁▁
MAE_validate,▄▇▅▃▇▇▅▄██▃▂▆▄▄▄▃▃▄▃▄▆▂▄▄▅▅▄▃▂▃▂▇▅▄▁▂▁▅▂
MAPE_test,▁
MAPE_train,▂▁▂▁▂▁▁▂▂▂▆▂▂▁▄▂▂▃▂▁▁▁▂▂▂▁█▂▂▂▂▁▁▁▁▁▁▁▁▁
MAPE_validate,███▃▄▃▃▅▇▄▃▃▄▂▂▂▂▂▄▂▄▄▅▂▆▂▄▂▂▄▂▁▂▄▃▂▁▃▂▅
MSE_test,▁
MSE_train,▄▁█▁▂▂▁▄▆▇▄▃▂▁▂▂▃▃▄▁▁▂▂▃▁▁▂▅▇█▃▃▁▂▁▁▃▁▁▁


All experiments completed.


In [11]:

# Iterate through each dataset
for dataset_name, data in datasets.items():
    # Iterate through each kernel
    train,valid ,test = split_data(data, train_ratio= 0.8,valid_ratio=0)
    
    for kernel_name, kernel in kernels.items():
        # Initialize the Gaussian Process Regressor with the current kernel
        gpr = GaussianProcessRegressor(kernel=kernel)

        # Iterate through each feature
        for feature_name, feature_data in features.items():
            if wandb.run is not None:
                wandb.run.finish()
                
            run = wandb.init(project="Merged_GPR_Experiments",
                             name=f"Expand_{dataset_name}_{kernel_name}:{feature_name}")
            
            try:
                
                # Configuration parameters (this could be extended depending on what you'd like to track)
                config = wandb.config
                # config.ini_train_win_len = 24
                config.ini_train_win_len = 24 * 7
                config.valid_win_len = 24
                # config.valid_win_len = 24 *7
                config.step_size = 1 # 24 hours or 1 day

                # Call the expanding window function
                y_pred, y_std, gpr = expanding_window_gpr_process(
                                                                run,
                                                                train,
                                                                gpr,
                                                                test,
                                                                ini_train_win_len=config.ini_train_win_len, 
                                                                valid_win_len=config.valid_win_len,
                                                                features=feature_data,
                                                                target='centered_car_count',
                                                                agg_func='mean'
                                                                )

                # Log results and close the wandb run
                # wandb.log({})
            except Exception as e:
                print(f"An error occurred: {e}")
            finally:
                run.finish()

print('All experiments completed.')


Processing expanding window:   0%|                                          | 0/108 [00:00<?, ?it/s]/home/sf_afn/usr_sa/anaconda3/envs/prkng_v2_env/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:429: ConvergenceWarning:

The optimal value found for dimension 0 of parameter k1__k1__k1__k2__length_scale is close to the specified upper bound 1000.0. Increasing the bound and calling fit again may find a better value.

Processing expanding window:   2%|▋                                 | 2/108 [00:00<00:22,  4.77it/s]

Step: 1


Processing expanding window:   4%|█▎                                | 4/108 [00:01<00:27,  3.85it/s]/home/sf_afn/usr_sa/anaconda3/envs/prkng_v2_env/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:429: ConvergenceWarning:

The optimal value found for dimension 0 of parameter k1__k1__k1__k2__length_scale is close to the specified upper bound 1000.0. Increasing the bound and calling fit again may find a better value.



Step: 2


Processing expanding window:   6%|█▉                                | 6/108 [00:01<00:27,  3.72it/s]

...

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Processing expanding window:   0%|                                          | 0/108 [00:00<?, ?it/s]


An error occurred: 'numpy.ndarray' object has no attribute 'predict'


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


All experiments completed.
